##### 2019 텐서플로우에서 저술한 저작권입니다.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/custom_callback"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />텐서플로우에서 보기.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃에서 소스 보기</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />노트북에 다운</a>
  </td>
</table>

# 케라스 커스텀 callbacks
커스텀 callback은 케라스 모델을 읽고 변경하는 것을 포함하여 교육,평가 또는 추론 중에 케라스 모델의 동작을 사용자 정의를 하는 강력한 도구 입니다. 예시로는 트레이닝 과정과 결과를 텐서보드를 통해 표현하고 시각화 하는 텐서플로우.callbacks.텐서보드 또는, 모델이 트레이닝중에 또는 그 이상에서 자동적으로 저장되는 텐서플로우.callbacks.모델.Checkpoint를 포함합니다. 이안내서에서는 케라스 callback이 무엇인지, 언제 불릴지, 무엇을 할 수 있는지, 그리고 어떨게 여러분 자신의 callback을 시작하기 위해 몇 가지 간단한 callback 애플리케이션을 생성하는 데모가 있을 것입니다.

## 배열

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %코랩에만 존재하는 텐서플로우_버전.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

## 케라스 callbacks 도입
케라스에서 'Callback'은 특정한 기능성을 제공하기 위해 하위로 분류되는 피톤 class로 트레이닝의 다양한 단계 (batch/ epoch 시작및 종료를 포함합니다), 시험 및 예측에서 호출되는 일련의 방법을 포함합니다. Callback은 훈련 중에 모델의 내부 상태와 통계를 보기 위해 유용합니다. '테너플로우.케라스.모델.적합()', '텐서플로우.케랏.모델.평가()' 그리고 '텐서플로우.케라스.모델.예상()' 방법으로 callbacks 리스트(callbacks라는 키워드로)를 전달할 수 있습니다. Callback의 방법은 교육/평가/추론의 다른 단계에서 호출됩니다.

시작하려면, 텐서플로우를 가져오고 간단한 순차적 케라스 모델을 정의해봅니다:

In [0]:
# callbacks를 추가하기 위한 케라스 정의
def get_model():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(1, activation = 'linear', input_dim = 784))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1), loss='mean_squared_error', metrics=['mae'])
  return model

그런다음 케라스 데이터 세트 API에서 교육및 테스트를 위한 최소 데이터를 로드하시면 됩니다.:

In [0]:
# 최소데이터와 이것의 사전처리의 예시를 로딩합니다.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

이제 모든 데이터의 batch의 시작과 끝을 추적할 수 있는 간단한 커스텀 callback을 정의하면 됩니다. 그 동안 이것은 현재 batch의 인덱스를 프린트 합니다.

In [0]:
import datetime

class MyCustomCallback(tf.keras.callbacks.Callback):

  def on_train_batch_begin(self, batch, logs=None):
    print('Training: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

  def on_train_batch_end(self, batch, logs=None):
    print('Training: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

  def on_test_batch_begin(self, batch, logs=None):
    print('Evaluating: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

  def on_test_batch_end(self, batch, logs=None):
    print('Evaluating: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

텐서플로우.케라스.모델.적합()과 같은 모델 메소드를 callback에 제공하는 것은 해당 단계에서 메소드가 호출되도록 보장합니다:

In [0]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          epochs=1,
          steps_per_epoch=5,
          verbose=0,
          callbacks=[MyCustomCallback()])

##callbacks을 수행하는 모델 방법
사용자는 다음 텐서플로우.케랏.모델 방법을 따라 callback 리스트를 제공합니다:
#### [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
일정한 수의 epochs(데이터 집합에 대한 반복, 또는 파이썬 생성기에 의한 batch별 데이터 산출)에 대한 모델을 추적합니다..
#### [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
주어진 데이터 또는 데이터 생성기에 대한 모델을 평가합니다. 평가에서 손실 및 메트릭값을 출력합니다.
#### [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
입력 데이터 또는 데이터 생성기에 대한 출력을 예측 생성 합니다.


In [0]:
_ = model.evaluate(x_test, y_test, batch_size=128, verbose=0, steps=5,
          callbacks=[MyCustomCallback()])

## callback 방법의 개요


### 교육/테스트/예측을 위한 일반적인 방법
트레이닝, 테스트 및 예측을 위해 재정의할 수 있는 방법은 다음과 같습니다.
#### `on_(train|test|predict)_begin(self, logs=None)`
'적합성'/'평가'/'예측'의 시작을 호출합니다.
#### `on_(train|test|predict)_end(self, logs=None)`
'적합성'/'평가'/'예측'의 끝을 호출합니다.
#### `on_(train|test|predict)_batch_begin(self, batch, logs=None)`
트레이닝/시험/예측이 진행되는 동안 batch의 진행 직전에 호출됩니다. 이 메소드 내에서, 일지는 batch의 현재 수와 사이즈를 나타내는 자유로운 크기의 키로 받아 써집니다.
#### `on_(train|test|predict)_batch_end(self, batch, logs=None)`
batch의 트레이닝/시험/예측의 끝에서 호출됩니다. 이 메소드에서, 로그는 상태 저장 메트릭 결과를 포함하는 받아쓰기입니다..

### 특정한 메소드 트레이닝
또한, 교육에 대해서는 다음 사항이 제공됩니다.
#### on_epoch_begin(self, epoch, logs=None)
트레이닝 동안의 epoch의 시작에서 호출됩니다.
#### on_epoch_end(self, epoch, logs=None)
트레이닝 동안의 epoch의 끝에서 호출됩니다.


### 로그 사용 설명
로그는 손실값과 batch 또는 epoch의 끝에 모든 메트릭스를 포함합니다. 예제에는 손실과 과실을 의미하는 것을 포함합니다.

In [0]:
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):

  def on_train_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_test_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_epoch_end(self, epoch, logs=None):
    print('The average loss for epoch {} is {:7.2f} and mean absolute error is {:7.2f}.'.format(epoch, logs['loss'], logs['mae']))

model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=3,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback()])

유사하게 평가()의 호출에서 callbacks를 제공할 수 있습니다.

In [0]:
_ = model.evaluate(x_test, y_test, batch_size=128, verbose=0, steps=20,
          callbacks=[LossAndErrorPrintingCallback()])

## 케라스 callback 어플리케이션의 예시
다음 분야에서는 간단한 callback 어플리케이션을 생성하는 방법을 안내합니다.

### 최소 손실의 조기 정지
첫번째 예시는 손실의 최소화가 모델.중지_트레이닝(부울)의 변형에 의해 도달할때 케라스 트레이닝을 멈추는 callback의 생성을 보여줍니다. 선택적으로, 사용자는 트레이닝이 끝날 때까지 트레이닝이 기다려야하는 epoch의 수를 명시하기 위해 논쟁을 위한 인내를 제공 할 수 있습니다.

`텐서플로우.케라스.callbacks.조기중지`는 좀 더 완성적이고 일반적인 실행을 제공합니다.

In [0]:
import numpy as np

class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):
  """손실이 최소 수준일 때 훈련을 중지합니다. 즉, 손실이 감소하는 것을 멈춥니다.

  논쟁:
      인내: 최소가 발생한 후 대기할 epochs의 수
      트레이닝 중지와 향상 없음의 수 이후에.
  """

  def __init__(self, patience=0):
    super(EarlyStoppingAtMinLoss, self).__init__()

    self.patience = patience

    # 최소 손실을 발생하고 가중치를 저장하는 최선의 무게.
    self.best_weights = None

  def on_train_begin(self, logs=None):
    # 손실이 더 이상 최소가 아닐 때 기다려온 epoch의 수.
    self.wait = 0
    # 트레이닝이 멈춘 시점의 epoch.
    self.stopped_epoch = 0
    # 최고를 무한으로 초기화.
    self.best = np.Inf

  def on_epoch_end(self, epoch, logs=None):
    current = logs.get('loss')
    if np.less(current, self.best):
      self.best = current
      self.wait = 0
      # 현재 결과가 더 나은 경우(낮은 경우)최고의 가중치를 기록.
      self.best_weights = self.model.get_weights()
    else:
      self.wait += 1
      if self.wait >= self.patience:
        self.stopped_epoch = epoch
        self.model.stop_training = True
        print('Restoring model weights from the end of the best epoch.')
        self.model.set_weights(self.best_weights)

  def on_train_end(self, logs=None):
    if self.stopped_epoch > 0:
      print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))

In [0]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=30,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()])

### 비율 스케쥴링 학습

모델 훈련에서 흔히 행해지는 한 가지 일은 더 많은 epochs가 지나감에 따라 학습 속도를 바꾸는 것입니다. 케라스 백엔드는 변수를 설정하는데 사용할 수 있는 겟_벨류 API를 노출합니다. 이 예에서 사용자 지정 콜백을 사용하여 학습 속도를 동적으로 변경하는 방법을 보여드리고 있습니다.

노트: 이것은 더 일반적인 실행을 위한 callbacks러닝레이드스케쥴러와 케라스옵티마이져스케쥴러를 보는 한 예시입니다.

In [0]:
class LearningRateScheduler(tf.keras.callbacks.Callback):
  """스케쥴에 따라 학습속도를 설정하는 러닝레이트스케쥴러.

  논쟁:
      스케쥴러: epoch의 인덱스를 가져오는 함수
          (정수, 0의 인덱스) 와 현재 학습 비율
          입력과 출력에 따른 새로운 학습 비율의 산출 (플로트).
  """

  def __init__(self, schedule):
    super(LearningRateScheduler, self).__init__()
    self.schedule = schedule

  def on_epoch_begin(self, epoch, logs=None):
    if not hasattr(self.model.optimizer, 'lr'):
      raise ValueError('Optimizer must have a "lr" attribute.')
    # 모델 옵티마이져에서 현재 학습속도 확인.
    lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
    # 스케쥴 학습 비율을 얻기 위한 스케쥴 함수의 호출.
    scheduled_lr = self.schedule(epoch, lr)
    # epoch 시작 전의 값으로 돌린 후 옵티마이져에 설정
    tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
    print('\nEpoch %05d: Learning rate is %6.4f.' % (epoch, scheduled_lr))

In [0]:
LR_SCHEDULE = [
    # (epoch의 시작, 학습 비율) 투플
    (3, 0.05), (6, 0.01), (9, 0.005), (12, 0.001)
]

def lr_schedule(epoch, lr):
  """epoch을 기반으로 한 스케쥴 학습 비율을 검색하는 도우미 함수."""
  if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
    return lr
  for i in range(len(LR_SCHEDULE)):
    if epoch == LR_SCHEDULE[i][0]:
      return LR_SCHEDULE[i][1]
  return lr

model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=15,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), LearningRateScheduler(lr_schedule)])

### 케라스 callbacks 표준
[API doc를 방문](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks). 하여 기존 케라스 callbacks를 확인하시면 됩니다. 어플리케이션에는 csv로깅, 모델저장, 텐서보드에서의 시각화 등이 포함됩니다.